In [ ]:
from acr.utils import SOM_BLUE, ACR_BLUE, NNXR_GRAY

MAIN_EXP = 'swi'
SUBJECT_TYPE = 'acr'
MAIN_COLOR = SOM_BLUE if SUBJECT_TYPE == 'som' else ACR_BLUE

In [ ]:
#-------------------------- Standard Imports --------------------------#
%reload_ext autoreload
%autoreload 2
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import acr
import pingouin as pg
from scipy.stats import shapiro, normaltest
import numpy as np

import warnings
warnings.filterwarnings('ignore')

probe_ord = ['NNXr', 'NNXo']
hue_ord = [NNXR_GRAY, MAIN_COLOR]

from acr.plots import lrg
plt.rcdefaults()
lrg()
plt.rcParams['xtick.bottom'] = False
#--------------------------------- Import Publication Functions ---------------------------------#
pu = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/pub_utils.py', 'pub_utils')
dag = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/data_agg.py', 'data_agg')

In [ ]:
notebook_figure_root = f'{acr.utils.PAPER_FIGURE_ROOT}/schems'

In [ ]:
from acr.utils import sub_swi_exps

In [ ]:
len(sub_swi_exps)

In [ ]:
# covers all SWI, ACR, and Control mice
full_hyps = {}
hyp_dicts = {}
from acr.utils import sub_swi_exps
for subject in sub_swi_exps:
    exp = sub_swi_exps[subject][0]
    full_hyps[subject] = acr.io.load_hypno_full_exp(subject, exp)
    #hyp_dicts[subject] = acr.hypnogram_utils.create_acr_hyp_dict(subject, exp, true_stim=False)
full_hyps['ACR_20'] = acr.io.load_hypno_full_exp('ACR_20', 'swisin')

In [ ]:
sleep_states = ['NREM', 'REM', 'Transition-to-REM', 'Transition-to-NREM']
bl_pct_sleep = []
sd_pct_sleep = []
for sub in full_hyps:
    bl_start = full_hyps[sub].start_time.min().replace(hour=9, minute=0, second=0, microsecond=0)
    bl_period_hyp = full_hyps[sub].trim_select(bl_start, bl_start + pd.Timedelta(hours=5))
    print(sub, bl_period_hyp.duration.sum().total_seconds())
    if bl_period_hyp.duration.sum().total_seconds() < 2:
        continue
    bl_sleep = bl_period_hyp.keep_states(sleep_states)
    bl_pct_sleep.append(bl_sleep.duration.sum().total_seconds() / bl_period_hyp.duration.sum().total_seconds())
    
    sd_start = bl_start + pd.Timedelta('24h')
    sd_period_hyp = full_hyps[sub].trim_select(sd_start, sd_start + pd.Timedelta(hours=5))
    print(sub, sd_period_hyp.duration.sum().total_seconds())
    sd_sleep = sd_period_hyp.keep_states(sleep_states)
    sd_pct_sleep.append(sd_sleep.duration.sum().total_seconds() / sd_period_hyp.duration.sum().total_seconds())
    print(sd_pct_sleep[-1])

In [ ]:
from acr.utils import NREM_RED

In [ ]:
plt.rcdefaults()
acr.plots.supl()

In [ ]:
fig_id = 'sd_effectiveness'
fig_name = f'All-Subs__{fig_id}'
fig_path = f'{notebook_figure_root}/{fig_name}.png'
f, ax = plt.subplots(1, 1, figsize=(5, 7))
acr.plots.add_boxplot(ax, bl_pct_sleep, [0.3], color=NREM_RED, alpha=0.8)
acr.plots.add_data_points(ax, bl_pct_sleep, 0.36, color=NREM_RED, alpha=0.8, s=120)

acr.plots.add_boxplot(ax, sd_pct_sleep, [0.7], color=NREM_RED, alpha=0.8)
acr.plots.add_data_points(ax, sd_pct_sleep, 0.64, color=NREM_RED, alpha=0.8, s=120)

acr.plots.add_paired_lines(ax, bl_pct_sleep, sd_pct_sleep, 0.36, 0.64, colors=[NREM_RED, NREM_RED], alpha=0.65, linewidth=4)
ax.set_xlim(0.25, 0.75)
ax.set_xticklabels(['Baseline', 'Sleep Deprivation'])
plt.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
# =============================
# ========== STATS ============
# =============================
write = True

diffs = nnxr - nnxo
nnxr = np.array(bl_pct_sleep)
nnxo = np.array(sd_pct_sleep)
shap_stat, shap_p = shapiro(diffs) # test the paired differences for normality
agostino_stat, agostino_p = normaltest(diffs) # test the paired differences for normality
print(f'shapiro_p-value: {shap_p}')
print(f'd,agostino_p-value: {agostino_p}')

#stats = pg.ttest(nnxr, nnxo, paired=True)
stats = pg.wilcoxon(nnxr, nnxo)

hg = pg.compute_effsize(nnxr, nnxo, paired=True, eftype='hedges')
print(f'hedges g: {hg}')

r = acr.stats.calculate_wilx_r(stats['W-val'][0], len(nnxr))

if write:
    # ==== Write Stats Results ====
    stats_name = f'{fig_name}'
    acr.stats.write_stats_result(stats_name, 'wilcoxon', stats['W-val'][0], stats['p-val'][0], 'r, RBC', [r, stats['RBC'][0]])
    
    # ===== Write Source Data =====
    source_data = pd.DataFrame({'contra_control': nnxr, 'off_induction': nnxo, 'subject': np.arange(len(nnxr))})
    pu.write_source_data(source_data, stats_name)
stats

# Example

In [ ]:
sub = 'ACR_40'
bl_start = full_hyps[sub].start_time.min().replace(hour=9, minute=0, second=0, microsecond=0)
bl_period_hyp = full_hyps[sub].trim_select(bl_start, bl_start + pd.Timedelta(hours=5))
print(sub, bl_period_hyp.duration.sum().total_seconds())

bl_sleep = bl_period_hyp.keep_states(sleep_states)

sd_start = bl_start + pd.Timedelta('24h')
sd_period_hyp = full_hyps[sub].trim_select(sd_start, sd_start + pd.Timedelta(hours=5))
print(sub, sd_period_hyp.duration.sum().total_seconds())
sd_sleep = sd_period_hyp.keep_states(sleep_states)

In [ ]:
f, ax = kde.plot.main.plot_basic_hypnogram(bl_period_hyp, style_path='/home/kdriessen/gh_master/acr/acr/plot_styles/acr_pub.mplstyle', single_tone=True, size=(15, 3))
fig_path = f'{notebook_figure_root}/bl_hypnogram_demo.png'
plt.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
f, ax = kde.plot.main.plot_basic_hypnogram(sd_period_hyp, style_path='/home/kdriessen/gh_master/acr/acr/plot_styles/acr_pub.mplstyle', single_tone=True, size=(15, 3))
fig_path = f'{notebook_figure_root}/sd_hypnogram_demo.png'
plt.savefig(fig_path, dpi=600, bbox_inches='tight', transparent=True)